In [10]:
from dotenv import load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import sys
sys.path.append("../scripts/")  # Add the scripts folder to the path
from load_data import load_data_from_postgres, load_data_using_sqlalchemy
from data_preprocessing import clean_data, handle_outliers
from data_transform import convert_bytes_to_mb, rename_columns
from engagement_metrics import calculate_engagement_metrics, normalize_metrics

In [11]:
# Load environment variables from .env file
load_dotenv()

# Fetch database connection parameters from environment variables
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

In [12]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"

# Load data from PostgreSQL using SQLAlchemy
df = load_data_using_sqlalchemy(query)

# Display the first few rows of the dataframe
if df is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")

Successfully loaded the data


In [13]:
telecom_data = clean_data(df)
print(telecom_data.info())
print(telecom_data.isnull().sum())  # Ensure there are no missing values
print(telecom_data.shape)          # Check the shape matches (148915, 44)


<class 'pandas.core.frame.DataFrame'>
Index: 148488 entries, 0 to 149999
Data columns (total 44 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Bearer Id                       148488 non-null  object        
 1   Start                           148488 non-null  datetime64[ns]
 2   Start ms                        148488 non-null  float64       
 3   End                             148488 non-null  datetime64[ns]
 4   End ms                          148488 non-null  float64       
 5   IMSI                            148488 non-null  object        
 6   MSISDN/Number                   148488 non-null  object        
 7   IMEI                            148488 non-null  object        
 8   Last Location Name              148488 non-null  category      
 9   Avg RTT DL (ms)                 148488 non-null  float64       
 10  Avg RTT UL (ms)                 148488 non-null  float64     

In [14]:
# Select numeric columns
numeric_columns = telecom_data.select_dtypes(include=['float64', 'int64']).columns

# Treat outliers using the handle_outliers function
telecom_data = handle_outliers(telecom_data, numeric_columns)


In [17]:
# Define the renaming dictionary
rename_dict = {'Dur. (ms).1': 'Dur. (ms)'}

# Rename columns using the rename_columns function
telecom_data = rename_columns(telecom_data, rename_dict)

# Verify the column names
print(df.columns)


Index(['Bearer Id', 'Start', 'Start ms', 'End', 'End ms', 'Dur. (ms)', 'IMSI',
       'MSISDN/Number', 'IMEI', 'Last Location Name', 'Avg RTT DL (ms)',
       'Avg RTT UL (ms)', 'Avg Bearer TP DL (kbps)', 'Avg Bearer TP UL (kbps)',
       'TCP DL Retrans. Vol (Bytes)', 'TCP UL Retrans. Vol (Bytes)',
       'DL TP < 50 Kbps (%)', '50 Kbps < DL TP < 250 Kbps (%)',
       '250 Kbps < DL TP < 1 Mbps (%)', 'DL TP > 1 Mbps (%)',
       'UL TP < 10 Kbps (%)', '10 Kbps < UL TP < 50 Kbps (%)',
       '50 Kbps < UL TP < 300 Kbps (%)', 'UL TP > 300 Kbps (%)',
       'HTTP DL (Bytes)', 'HTTP UL (Bytes)', 'Activity Duration DL (ms)',
       'Activity Duration UL (ms)', 'Dur. (ms).1', 'Handset Manufacturer',
       'Handset Type', 'Nb of sec with 125000B < Vol DL',
       'Nb of sec with 1250B < Vol UL < 6250B',
       'Nb of sec with 31250B < Vol DL < 125000B',
       'Nb of sec with 37500B < Vol UL',
       'Nb of sec with 6250B < Vol DL < 31250B',
       'Nb of sec with 6250B < Vol UL < 37500B',
